In [1]:
from collections import namedtuple

#Same as tuple but the fields are named for convenience
#this says we have four fields
OneWord=namedtuple("OneWord",["word","pos_label","chunk_label","entity_label"])

def read_conll2003(f_name):
    """Yield complete sentences"""
    current_sentence=[] #This will be a list of (word,label), which we accumulate for each sentence
    with open(f_name) as f:
        for line in f:
            line=line.strip() #drop whitespace
            if line.startswith("-DOCSTART-"): #let's not worry about these for the time being
                continue
            if not line: #sentence break
                if current_sentence: #if we gathered a sentence, we should yield it, because a new one starts
                    yield current_sentence #much like return, but continues past this line once the element has been consumed
                    current_sentence=[] #...and start a new one
                continue
            #if we made it here, we are on a normal line
            columns=line.split() #an actual word line
            assert len(columns)==4 #we should have four columns, looking at the data
            current_sentence.append(OneWord(*columns)) #* expands columns as arguments to OneWord constructor
        else: #for ... else -> the else part is executed once, when "for" runs out of elements
            if current_sentence: #yield also the last one!
                yield current_sentence

#Now just read the data in
sentences_train=list(read_conll2003("dataset/train.txt"))
sentences_dev=list(read_conll2003("dataset/valid.txt"))

print("First three sentences")
for sent in sentences_dev[:3]:
    print(sent)
    print()

First three sentences
[OneWord(word='CRICKET', pos_label='NNP', chunk_label='B-NP', entity_label='O'), OneWord(word='-', pos_label=':', chunk_label='O', entity_label='O'), OneWord(word='LEICESTERSHIRE', pos_label='NNP', chunk_label='B-NP', entity_label='B-ORG'), OneWord(word='TAKE', pos_label='NNP', chunk_label='I-NP', entity_label='O'), OneWord(word='OVER', pos_label='IN', chunk_label='B-PP', entity_label='O'), OneWord(word='AT', pos_label='NNP', chunk_label='B-NP', entity_label='O'), OneWord(word='TOP', pos_label='NNP', chunk_label='I-NP', entity_label='O'), OneWord(word='AFTER', pos_label='NNP', chunk_label='I-NP', entity_label='O'), OneWord(word='INNINGS', pos_label='NNP', chunk_label='I-NP', entity_label='O'), OneWord(word='VICTORY', pos_label='NN', chunk_label='I-NP', entity_label='O'), OneWord(word='.', pos_label='.', chunk_label='O', entity_label='O')]

[OneWord(word='LONDON', pos_label='NNP', chunk_label='B-NP', entity_label='B-LOC'), OneWord(word='1996-08-30', pos_label='CD',

In [2]:
def generate_sentence_features(sent):
    #Given a sentence as a list of (word, label) pairs
    #generate the features for every word
    #The result should be a list of same length as the sentence
    #Each item is a dictionary of {"feature name"->feature value} mappings, holding all features of the word at that position
    
    sent_features=[] #this will be the result
    for one_word in sent:
        #We must do nothing with label
        #it just happens to be around
        word_features={}
        word_features["word_"+one_word.word]=1 #the word itself is a feature
        sent_features.append(word_features)
    return sent_features

print(generate_sentence_features(sentences_dev[0])  )

[{'word_CRICKET': 1}, {'word_-': 1}, {'word_LEICESTERSHIRE': 1}, {'word_TAKE': 1}, {'word_OVER': 1}, {'word_AT': 1}, {'word_TOP': 1}, {'word_AFTER': 1}, {'word_INNINGS': 1}, {'word_VICTORY': 1}, {'word_.': 1}]


In [3]:
#...now we can generate the training examples
def prep_data(sentences):
    all_labels=[] #here we gather labels for all words in all sentences
    all_features=[] #here we gather features for all words in all sentences
    for sentence in sentences:
        sent_features=generate_sentence_features(sentence)
        assert len(sent_features)==len(sentence)
        #Now we can get, for every position its label and its features
        for one_word,features in zip(sentence,sent_features):
            all_labels.append(one_word.pos_label) #label
            all_features.append(features)         #and features to go with it
    return all_labels, all_features

train_labels,train_features=prep_data(sentences_train)
dev_labels,dev_features=prep_data(sentences_dev)

In [4]:
from sklearn.feature_extraction import DictVectorizer
vectorizer=DictVectorizer()
vectorizer.fit(train_features)
print("Vectorizer vocab size:",len(vectorizer.vocabulary_))

feature_vectors_train=vectorizer.transform(train_features)
feature_vectors_dev=vectorizer.transform(dev_features)

print("Train shape",feature_vectors_train.shape)
print("Dev shape",feature_vectors_dev.shape)

Vectorizer vocab size: 23623
Train shape (203621, 23623)
Dev shape (51362, 23623)


In [5]:
import sklearn.svm

classifier=sklearn.svm.LinearSVC(C=0.05,verbose=1)
classifier.fit(feature_vectors_train, train_labels)

c:\Users\ajays\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


[LibLinear]

LinearSVC(C=0.05, verbose=1)

In [6]:
classifier.score(feature_vectors_dev,dev_labels)

0.8655426190568903

In [7]:
def generate_sentence_features(sent):
    #Given a sentence as a list of (word, label) pairs
    #generate the features for every word
    #The result should be a list of same length as the sentence
    #Each item is a dictionary of {"feature name"->feature value} mappings, holding all features of the word at that position
    
    sent_features=[] #this will be the result
    for word_idx, one_word in enumerate(sent):
        #We do nothing with label
        #it just happens to be around
        word_features={}
        word_features["word_"+one_word.word]=1 #the word itself is a feature
        if word_idx!=0:
            word_features["left_word_"+sent[word_idx-1].word]=1
        if word_idx!=len(sent)-1:
            word_features["right_word_"+sent[word_idx+1].word]=1
        sent_features.append(word_features)
    return sent_features

train_labels,train_features=prep_data(sentences_train)
dev_labels,dev_features=prep_data(sentences_dev)
vectorizer=DictVectorizer()
vectorizer.fit(train_features)
feature_vectors_train=vectorizer.transform(train_features)
feature_vectors_dev=vectorizer.transform(dev_features)

print("Train shape",feature_vectors_train.shape)
print("Dev shape",feature_vectors_dev.shape)

classifier=sklearn.svm.LinearSVC(C=1,verbose=1)
classifier.fit(feature_vectors_train, train_labels)
classifier.score(feature_vectors_dev,dev_labels)

Train shape (203621, 68467)
Dev shape (51362, 68467)
[LibLinear]

c:\Users\ajays\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


0.9292862427475566

In [8]:
# Let us try to look at some predictions
sentence="I can house you in my house .".split()

sentence_data=[OneWord(w,"XXX","XXX","XXX") for w in sentence] #we need to fake this a bit, to get data in the correct format
_,sentence_features=prep_data([sentence_data])
sentence_vectors=vectorizer.transform(sentence_features)
predictions=classifier.predict(sentence_vectors)
for word,label in zip(sentence,predictions):
    print(word,label)


I PRP
can MD
house VB
you PRP
in IN
my PRP$
house NN
. .


In [9]:
print("Learned coefficients:",classifier.coef_.shape)
print("Classes in the data:",classifier.classes_)


Learned coefficients: (45, 68467)
Classes in the data: ['"' '$' "''" '(' ')' ',' '.' ':' 'CC' 'CD' 'DT' 'EX' 'FW' 'IN' 'JJ' 'JJR'
 'JJS' 'LS' 'MD' 'NN' 'NNP' 'NNPS' 'NNS' 'NN|SYM' 'PDT' 'POS' 'PRP' 'PRP$'
 'RB' 'RBR' 'RBS' 'RP' 'SYM' 'TO' 'UH' 'VB' 'VBD' 'VBG' 'VBN' 'VBP' 'VBZ'
 'WDT' 'WP' 'WP$' 'WRB']


In [10]:
import numpy

#Reverse the dictionary
index2feature={}
for feature,idx in vectorizer.vocabulary_.items():
    assert idx not in index2feature #This really should hold
    index2feature[idx]=feature
#Now we can query index2feature to get the feature names as we need

i=list(classifier.classes_).index("NN") #which of the coefficients corresponds to nouns?
indices=numpy.argsort(classifier.coef_[i])
print("Negative features")
for idx in indices[:30]:
    print(index2feature[idx])
print("-------------------------------")
print("Positive features")
for idx in indices[::-1][:30]: #you can also do it the other way round, reverse, then pick
    print(index2feature[idx])

Negative features
left_word_will
left_word_Sale
word_,
left_word_going
left_word_could
left_word_would
left_word_goals
right_word_A-rated
left_word_We
word_and
left_word_At
word_in
left_word_still
right_word_announcement
left_word_mixer
left_word_can
left_word_I
left_word_should
left_word_kms
left_word_might
left_word_8:00
left_word_prices
left_word_Mike
right_word_SCOREBOARD
left_word_must
left_word_n't
left_word_overs
right_word_effect
left_word_Services
word_two
-------------------------------
Positive features
word_world
word_power
word_consumer
word_peace
word_number
word_hospital
word_vouch
word_cricket
word_procure
word_soccer
word_victory
word_championship
word_staff
word_motor
word_value
word_cabinet
word_lunch
word_rain
word_injury
word_league
word_anyone
word_UNION
word_weekend
word_edge
word_parliament
word_shutdown
word_division
word_cash
word_tournament
word_race
